First, we import yprov4ml, this will come in handy to create the provenance documents of our experiment.

In [ ]:
import yprov4ml

Here we define a simple objective function that is intended to be used only for the generation of provenance documents.


In [ ]:
def simple_objective_function(params):
    x = params.get('X', 0)

    yprov4ml.start_run(
        prov_user_namespace="www.example.org",
        experiment_name=f"{x}_experiment", 
        provenance_save_dir="simple_workflow_prov",
        save_after_n_logs=100,
        collect_all_processes=False, 
        disable_codecarbon=False, 
        metrics_file_type=yprov4ml.MetricsType.NETCDF,
    )

    yprov4ml.log_param("X", x, yprov4ml.Context.TRAINING)

    y = (x - 20) ** 2

    yprov4ml.log_param("Y", y, yprov4ml.Context.TRAINING)

    yprov4ml.end_run(
        create_graph=False,
        create_svg=False,
        crate_ro_crate=False
    )

    return {"Y": y}

In [ ]:
val = [8, 12, 17, 19, 7, 22, 39, 35, 25, 28, 32, 26]

for v in val:
    simple_objective_function({'X': v})


This is a copy of the previous function, the yprov4ml has been removed to avoid duplicate provenance information, since the library itself 
will automatically log the execution of the objective.

In [ ]:
def objective_function(params):
    x = params.get('X', 0)

    y = (x - 20) ** 2

    return {"Y": y}

In [ ]:
from library import Experiment, OptimizationParameters, SearchSpace, IntParameter, OptimizerConfig

We define the search space of our optimization experiment, in this case we have only one parameter, X, which is an integer that can take values between 0 and 40.

Next, we define the optimization parameters, in this case we want to minimize the metric Y, which is the output of our objective function.

Then, we define the optimizer configuration, in this case we use the default configuration of the library.

In [ ]:
search_space = SearchSpace(parameters=[IntParameter("X", 0, 40)])

opt_params = OptimizationParameters(
    input=search_space,
    output=["Y"],
    directions=["min"]
)

optimizer_config = OptimizerConfig(
    200, 10, "ucb", 0.5
)

We create an experiment object, which will be responsible for running the optimization process and logging the provenance information.

- path_to_prov is the path where the provenance documents are stored
- n_iter is the number of iterations of the optimization process, in this case we set it to 10, which means that the optimizer will evaluate 10 different configurations of the parameters.

Finally, we run the experiment and print the best configuration found by the optimizer.

In [ ]:
exp = Experiment(
    optimization_parameters=opt_params,
    optimizer_config=optimizer_config,
    path_to_prov="./simple_workflow_prov",
    n_iter=10
)

exp.optimize(objective_function)

In [ ]:
print(exp.best_configuration(metric_name="Y"))